In [1]:
from utils.read import readExcel

import pandas as pd

In [24]:
df = readExcel(
    "./../../upgrade_mandi/raw-sheets-dump/Upgrade 24 september 2025.xlsx",
    sheetName="Sheet1",
)

df.columns = [column.strip().lower() for column in df.columns]

df["indents"] = df["indents"].apply(int)

In [25]:
df.columns

Index(['date', 'city', 'store id', 'sku_id', 'store_name', 'vendor',
       'item_code', 'product_name', 'weight', 'indents', 'supplier id', 'cost',
       'mrp', 'po number', 'entity name'],
      dtype='object')

In [26]:
groupByProductName = df.groupby("item_code")

In [29]:
final = pd.DataFrame(
    {
        "product_name": groupByProductName["product_name"]
        .unique()
        .apply(lambda names: names[0]),
        "weight": groupByProductName["weight"]
        .unique()
        .apply(lambda weights: weights[0]),
        "indents": groupByProductName["indents"].sum().astype(int),
    }
)
final.reset_index(inplace=True, drop=True)
final.columns = ["Product_Name", "Weight", "Indents"]

In [30]:
final["Sr No"] = range(1, len(final) + 1)
final["Final"] = (
    final["Weight"]
    .apply(
        lambda weight: "".join(
            [char for char in weight if 48 <= ord(char) and ord(char) <= 57]
        )
    )
    .astype(int)
    * final["Indents"]
)
final = pd.concat(
    [
        final,
        pd.DataFrame(
            {
                "Sr No": [""],
                "Product_Name": [""],
                "Weight": [""],
                "Indents": [f'Total: {final["Indents"].sum().astype(int)}'],
                "Final": [""],
            }
        ),
    ]
)

In [32]:
final[["Sr No", "Product_Name", "Weight", "Indents", "Final"]].to_excel(
    "./../Final.xlsx",
    sheet_name="Final Report",
    index=False,
    engine="openpyxl",
    freeze_panes=(1, 0),
    columns=["Sr No", "Product_Name", "Weight", "Indents", "Final"],
)